In [13]:
import re, os
import pandas as pd

def read_recipe(cuisine, source):
    data_file = f"./src/{cuisine}/{cuisine}_{source}.md"
    with open(data_file, "r") as f:
        content = f.read()
    class_list = content.split("\n# ")
    class_array = []
    data_list = []
    class_array.append(class_list[0].split(" ", 2)[1:])
    for class1 in class_list[1:]:
        class_array.append(class1.split(" ", 1))
    for class_row in class_array:
        class1, menu_content = class_row
        for menu in menu_content.split("\n## ")[1:]:
            menu_name, menu_detail = menu.split(" ", 1)
            data_list.append((class1, menu_name, menu_detail))
    columns = ["class", "menu_name", "menu_detail"]
    df = pd.DataFrame(data=data_list, columns=columns)
    df["cuisine"] = cuisine
    df["source"] = source
    return df

def read_recipes(cuisine, sources):
    df_list = []
    for source in sources:
        df_list.append(read_recipe(cuisine, source))
    return pd.concat(df_list)


def save_md(df):
    cuisine = df["cuisine"].iloc[0]

    for menu_name in df["menu_name"].drop_duplicates():
        df1 = df.loc[df["menu_name"] == menu_name]
        class1 = df1["class"].iloc[0]
        md_content = ""
        for idx in df1.index:
            menu_detail, source = df1.loc[idx, ["menu_detail", "source"]]
            md_content += f"## {source}版本\n\n{menu_detail}\n\n"
        out_folder = f"./src/{cuisine}/{class1}"
        md_file = f"{out_folder}/{menu_name}.md"
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        with open(md_file, "w") as f:
            f.write(md_content)
        
    summary_content = ""
    for class1 in df["class"].drop_duplicates():
        df1 = df.loc[df["class"] == class1]
        out_folder = f"./src/{cuisine}/{class1}"
        index_file = f"{out_folder}/index.md"
        index_content = f"# {class1}\n\n"
        summary_content += f"- [{class1}](./{cuisine}/{class1}/index.md)\n"
        for menu_name in df1["menu_name"].drop_duplicates():
            index_content += f"1. [{menu_name}]({menu_name}.md)\n"
            summary_content += f"    - [{menu_name}](./{cuisine}/{class1}/{menu_name}.md)\n"
        with open(index_file, "w") as f:
            f.write(index_content)
    # print(summary_content)

In [15]:
cuisine_dict = {
    # "anhui":        ["中国名菜谱"],
    # "beijing":      ["胡丽妹"],
    # "guangdong":    ["北京饭店"],
    # "huaiyang":     ["北京饭店"],
    # "shanghai":     ["李伯荣家常菜"],
    # "shanghai":     ["李伯荣家常菜", "李伯荣宴席菜"],
    "sichuan":      ["北京饭店", "锦江宾馆", "陈松如"],
    # "qingzhen":     ["杨永和"],
    # "tanjia":       ["北京饭店"]
}
for cuisine in cuisine_dict.keys():
    sources = cuisine_dict[cuisine]
    df = read_recipes(cuisine, sources)
    save_md(df)